# DAggerを使用した模倣学習

- エキスパートの動作を模倣して学習するための一連の処理
- 使用環境：CartPole（ポールが倒れないようにカートを動かす制御タスク）
- 使用ライブラリ：imitation, stable-baselines3, gymnasium

In [43]:
import tempfile

import numpy as np
import gymnasium as gym
from stable_baselines3.common.evaluation import evaluate_policy

from imitation.algorithms import bc
from imitation.algorithms.dagger import SimpleDAggerTrainer  # DAgger用
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

## step1. 環境準備とランダムシードの設定
- CartPole(カートとポール)タスクの環境を作成
- "seals"は、SEALSベンチマーク環境で安定している

In [44]:
rng = np.random.default_rng(0)  # 再現性のある乱数生成器

env = make_vec_env(
    "seals:seals/CartPole-v0",  # 使用する環境
    rng=rng,  # 乱数生成器を指定
)

## Step2. 事前学習済みのエキスパートポリシーをロード
- Hugging Face上で公開されている、優れたPROアルゴリズムで学習されたポリシーをロード
    - https://huggingface.co/HumanCompatibleAI
    - これは「模倣するべきもの」（先生, エキスパート）

- エキスパートが優秀か確認する
- 通常、達成可能な最大値である500の報酬を取得できる

In [45]:
expert = load_policy(
    "ppo-huggingface",                  # 使用するポリシーの名前
    organization="HumanCompatibleAI",   # Hugging Faceの組織名
    env_name="seals-CartPole-v0",       # 環境名
    venv=env,                           # 環境を指定  
)

# エキスパートポリシーの性能を確認（行動模倣する価値があるか？）
# CartPole環境では最大報酬が500。これに近いスコアを出せていれば、十分に優秀。
reward, _ = evaluate_policy(expert, env, 10)  # 10エピソード評価
print(f"エキスパートの平均報酬: {reward:.2f}")

/home/lilin/anaconda3/envs/stable_baselines/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


エキスパートの平均報酬: 500.00


## step4. 初期のBCを使用した模倣学習モデル
- Behavior Cloning（BC）を使用して、教師あり学習ベースの模倣学習を行う
- BCでは、分布シフト問題が発生する可能性がある。

In [46]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,    # 観測空間
    action_space=env.action_space,              # 行動空間
    rng=rng,                                    # 乱数生成器    
    device="cpu",                               # デバイス（CPUまたはGPU）
)

## step4. DAggerによる模倣学習
- SimpleDAggerTrainer を使い、DAggerによる学習を8000ステップ実行
- DAggerでは学習中にエキスパートの正しい行動を再度聞きながらデータを追加し、改善

In [47]:
with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:  # 一時ディレクトリを作成
    print(f"一時保存ディレクトリ：{tmpdir}")

    # DAggerトレーナーの初期化
    dagger_trainer = SimpleDAggerTrainer(  
        venv=env,
        scratch_dir=tmpdir,
        expert_policy=expert,
        bc_trainer=bc_trainer,
        rng=rng,
    )

    print("トレーニング前のエキスパートポリシーの評価を行います...")
    reward_before, _ = evaluate_policy(dagger_trainer.policy, env, 10)  # トレーニング前の評価
    print(f"トレーニング前のエキスパートの平均報酬: {reward_before:.2f}")

    # DAggerのトレーニングを実行
    # 8000ステップのトレーニングを行う
    print("DAggerトレーニングを開始します...")
    dagger_trainer.train(8_000)

一時保存ディレクトリ：/tmp/dagger_example_afl42mec
トレーニング前のエキスパートポリシーの評価を行います...
トレーニング前のエキスパートの平均報酬: 101.40
DAggerトレーニングを開始します...


Saving the dataset (1/1 shards): 100%|██████████| 1/1 [00:00<00:00, 264.08 examples/s]
0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 72.5      |
|    loss           | 0.692     |
|    neglogp        | 0.693     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
| rollout/          |           |
|    return_max     | 44        |
|    return_mean    | 27.1      |
|    return_min     | 17        |
|    return_std     | 8.13      |
---------------------------------


500batch [00:01, 284.49batch/s]
Saving the dataset (1/1 shards): 100%|██████████| 1/1 [00:00<00:00, 277.13 examples/s]
0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000339 |
|    entropy        | 0.339     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 98        |
|    loss           | 0.326     |
|    neglogp        | 0.326     |
|    prob_true_act  | 0.767     |
|    samples_so_far | 32        |
| rollout/          |           |
|    return_max     | 422       |
|    return_mean    | 361       |
|    return_min     | 239       |
|    return_std     | 56.3      |
---------------------------------


466batch [00:01, 349.71batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000367 |
|    entropy        | 0.367     |
|    epoch          | 2         |
|    l2_loss        | 0         |
|    l2_norm        | 161       |
|    loss           | 0.256     |
|    neglogp        | 0.257     |
|    prob_true_act  | 0.797     |
|    samples_so_far | 16032     |
| rollout/          |           |
|    return_max     | 500       |
|    return_mean    | 500       |
|    return_min     | 500       |
|    return_std     | 0         |
---------------------------------


1000batch [00:03, 283.34batch/s]


- **上の結果の説明**<br>
- データセット保存ログ
    `Saving the dataset (1/1 shards): 100%|██████████| ...`
    - 各回、環境でのプレイデータをエキスパートから収集し、保存
    - shardはデータを分割して保存する単位

- 進捗ログ

| 項目                         | 意味                         |
| -------------------------- | -------------------------- |
| `batch_size`               | 一度に学習に使うサンプル数（32個）         |
| `loss`                     | モデルの誤差（低いほどよい）             |
| `neglogp`                  | 正解行動の負の対数確率（= ロスとほぼ同義）     |
| `prob_true_act`            | 正解行動を予測できた確率（高いほど良い）       |
| `entropy`                  | ポリシーのランダム性（0.693 = 完全ランダム） |
| `return_mean`              | モデルによるプレイ時の平均スコア           |
| `return_max`, `return_min` | プレイ中に得た最高・最低スコア            |
| `samples_so_far`           | 今までに使用した合計サンプル数            |
- 初回のデータセットを保存して学習したとき
    - `return_mean`（プレイ時の平均スコア）が27.1と低スコアであり、失敗することが多かった。
- その状態で「正しい行動」をエキスパートに確認
    - 学習者が経験した軌道上の状態に対して、エキスパートの行動を記録（データセットを保存）して新しい学習データに追加する
- 新たなデータセットで再度学習
    - 初回の`return_mean`は、361と先ほどよりも高スコア
    - 最終的な`return_mean`は500と最高得点を得られている


- 最終的に学習したポリシー（方策）を10回テストして、平均報酬を表示

In [48]:
reward, _ = evaluate_policy(dagger_trainer.policy, env, 10)
print("トレーニング後の報酬:", reward)

トレーニング後の報酬: 500.0


# トレーニング後のエージェントの動作を動画で可視化
- 学習させる前と後で動画を出力してみると面白いかも

In [49]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
import os

# 動画保存先ディレクトリ
video_dir = "./videos/"
os.makedirs(video_dir, exist_ok=True)

# render_mode を "rgb_array" に設定して環境を作成
video_env = DummyVecEnv([lambda: gym.make("CartPole-v1", render_mode="rgb_array")])

# VecVideoRecorder でラップ
video_env = VecVideoRecorder(
    video_env,
    video_folder=video_dir,
    record_video_trigger=lambda step: step == 0,
    video_length=500,
    name_prefix="dagger_cartpole",
)

# ビヘイビアクローンされたエージェントのポリシーを使って実行
obs = video_env.reset()
done = False
while not done:
    action, _ = dagger_trainer.policy.predict(obs)
    obs, _, dones, _ = video_env.step(action)
    done = dones[0]

# 動画を保存
video_env.close()
print(f"動画を {video_dir} に保存しました。")


Saving video to /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/DAgger/videos/dagger_cartpole-step-0-to-step-500.mp4
MoviePy - Building video /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/DAgger/videos/dagger_cartpole-step-0-to-step-500.mp4.
MoviePy - Writing video /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/DAgger/videos/dagger_cartpole-step-0-to-step-500.mp4



MoviePy - Done !
MoviePy - video ready /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/DAgger/videos/dagger_cartpole-step-0-to-step-500.mp4
動画を ./videos/ に保存しました。


In [50]:
from IPython.display import Video

# 動画ファイルのパス（例: bc_agent-step-0-to-step-500.mp4）
video_path = os.path.join(video_dir, "dagger_cartpole-step-0-to-step-500.mp4")
Video(video_path, embed=True)